# Knowledge Distillation on MNIST
Knowledge distillation is the process of transferring the higher performance of a more expensive model to a smaller one.  In this notebook, we will explore performing this process on MNIST.  To begin with, I have provided access to pre-trained model that is large, but performant.  The exact architecture is not relevant (although you can inspect this easily if you wish).  It is straightforward to load in pytorch with

In [1]:
import torch
device = 'cpu'

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(28**2,800)
        self.l2 = torch.nn.Linear(800,800)
        self.l3 = torch.nn.Linear(800,10)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.dropout3 = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.l1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.l3(x)
        return x
    
big_model = torch.load('pretrained_model.pt').to(device)

First, let's establish the baseline performance of the big model on the MNIST test set.  Of course we'll need acces to the MNIST test set to do this.  At the same time, let's also get our transfer set, which in this case will be a $n=10$k subset of the full MNIST training set (using a subset is helpful for speeding up training of distilled models, and also helps showcase some of the improved performance due to model distillation).   

In [2]:
from torchvision import transforms, datasets
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

dataset_train = datasets.MNIST('./data', train=True, download=True, transform=transform)

dataset_test = datasets.MNIST('../data', train=False, download=True, transform=transform)

# This is a useful function that I didn't know about before
first_10k = list(range(0, 10000))
dataset_transfer = torch.utils.data.Subset(dataset_train, first_10k)

batch_size = 32
num_workers = 4
transfer_loader = torch.utils.data.DataLoader(dataset_transfer,batch_size=batch_size,num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dataset_test,batch_size=batch_size,num_workers=num_workers)

C:\Users\zesau\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Here's a function that runs the big model in test mode and provides the number of correct examples

In [3]:
def test(model,test_loader):
    correct = 0
    counter = 0
    model.eval()
    with torch.no_grad():
        for data,target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            logits = model(data)
            pred = logits.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            counter += batch_size
    return correct

test(big_model,test_loader)

9833

We find that the big model gets 167 examples wrong (not quite as good as the Hinton paper, but who cares). 

Now we would like to perform knowledge distillation by training a smaller model to approximate the larger model's performance on the transfer set.  First, let's build a smaller model.  You may use whatever architecture you choose, but I found that using two hidden layers, each with 200 units along with ReLU activations (and no regularization at all) worked fine.

In [4]:
class SmallNet(torch.nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.l1 = torch.nn.Linear(28**2,200)
        self.l2 = torch.nn.Linear(200,10)
        
    def forward(self, x):
        x = self.l1(x)
        x = torch.relu(x)
        x = self.l2(x)
        # First linear 
        # Don't forget to put the right operations here too!
        return x
    
small_model = SmallNet()
small_model.to(device)

SmallNet(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (l2): Linear(in_features=200, out_features=10, bias=True)
)

**To establish a baseline performance level, train the small model on the transfer set**  

In [5]:
# I'm giving you this training function: you'll need to modify it below to do knowledge distillation
def train(model,train_loader,n_epochs):
    optimizer = torch.optim.Adam(model.parameters(),1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(n_epochs):
        avg_l = 0.0
        counter = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            optimizer.zero_grad()
            logits = model(data)
            L = loss_fn(logits,target)
            L.backward()
            optimizer.step()
            with torch.no_grad():
                avg_l += L
                counter += 1
        print(epoch,avg_l/counter)

train(small_model,transfer_loader,50)

0 tensor(0.4267)
1 tensor(0.2031)
2 tensor(0.1322)
3 tensor(0.0885)
4 tensor(0.0589)
5 tensor(0.0386)
6 tensor(0.0252)
7 tensor(0.0189)
8 tensor(0.0153)
9 tensor(0.0140)
10 tensor(0.0143)
11 tensor(0.0072)
12 tensor(0.0266)
13 tensor(0.0225)
14 tensor(0.0104)
15 tensor(0.0098)
16 tensor(0.0072)
17 tensor(0.0095)
18 tensor(0.0179)
19 tensor(0.0140)
20 tensor(0.0081)
21 tensor(0.0076)
22 tensor(0.0053)
23 tensor(0.0087)
24 tensor(0.0095)
25 tensor(0.0085)
26 tensor(0.0037)
27 tensor(0.0043)
28 tensor(0.0012)
29 tensor(0.0146)
30 tensor(0.0291)
31 tensor(0.0173)
32 tensor(0.0015)
33 tensor(0.0002)
34 tensor(9.3183e-05)
35 tensor(7.5222e-05)
36 tensor(6.3918e-05)
37 tensor(5.5280e-05)
38 tensor(4.8316e-05)
39 tensor(4.2469e-05)
40 tensor(3.7518e-05)
41 tensor(3.3190e-05)
42 tensor(2.9391e-05)
43 tensor(2.5999e-05)
44 tensor(2.3030e-05)
45 tensor(2.0386e-05)
46 tensor(1.8031e-05)
47 tensor(1.5990e-05)
48 tensor(1.4185e-05)
49 tensor(1.2579e-05)


**Evaluate the small model on the test set, and comment on its accuracy relative to the big model.**  As you might expect, the performance is relatively worse.  

In [8]:
test(small_model,test_loader)

9610

**The primary task of this notebook is now as follows: create a new training function similar to "train" above, but instead called "distill".**  "distill" should perform knowledge distillation as outlined in this week's paper.  It should accept a few additional arguments compared to train, namely the big model, the temperature hyperparameter, and a hyperparameter $\alpha$ that weights the relative magnitude of the soft target loss and the hard target loss.

In [9]:
distilled_model = SmallNet()
distilled_model.to(device)

# The body of this method is currently copied verbatim from the train method above: 
# you will need to modify it to utilize the big_model, temperature, and alpha values 
# to perform knowledge distillation
def distill(small_model,big_model,T,alpha,train_loader,n_epochs):
    
    optimizer = torch.optim.Adam(big_model.parameters(),1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()
    small_model.train()
    big_model.train()
    for epoch in range(n_epochs):
        avg_l = 0.0
        counter = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            optimizer.zero_grad()
            small_logits = small_model(data)
            big_logits = big_model(data)
            target = torch.softmax(big_logits)
            Lsmall = loss_fn(small_logits/T**2,target)
            Lbig = loss_fn(big_logits/T**2,target)
            L = Lbig-Lsmall
            L.backward()
            optimizer.step()
            with torch.no_grad():
                avg_l += L
                counter += 1
        print(epoch,avg_l/counter) 
distill(distilled_model,big_model,T,alpha,transfer_loader,50)

NameError: name 'T' is not defined

In [ ]:
def distillation(y, labels, big_scores, T, alpha):
    return nn.KLDivLoss()(F.log_softmax(y/T), F.softmax(big_scores/T)) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)

In [43]:
def train(epoch, model, loss_fn):
    model.train()
    big_model.eval()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        big_output = big_model(data)
        big_output = big_output.detach()
        # teacher_output = Variable(teacher_output.data, requires_grad=False) #alternative approach to load teacher_output
        loss = loss_fn(output, target, big_output, T=20.0, alpha=0.7)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


**Finally, test your distilled model (on the test set) and describe how it performs relative to both big and small models.**